In [72]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Keys, ActionChains
import matplotlib as plt

from log import logging

In [81]:
driver=webdriver.Chrome()

driver.get("https://www.aviasales.by/")

In [74]:
def open_window_popular_countries():
    ActionChains(driver)\
        .key_down(Keys.PAGE_DOWN)\
        .perform()
    time.sleep(0.1)
    clickable_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="selene"]/div/div[2]/div/noindex/div/div/div[3]/div/div[2]/button'))
    )

    clickable_element.click()

def get_all_popular_countries():
    countries = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "s__MvVzMn1eHc7dL22r"))
    )
    return countries

def open_window_all_cities():
    clickable_element = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="selene"]/div/div[2]/div/section/div/div/div[2]/button'))
    )

    clickable_element.click()

In [86]:
open_window_popular_countries()
len_list_with_countries = len(get_all_popular_countries()) - 1

count = 0

In [ ]:
while count <= len_list_with_countries:
    buttons = get_all_popular_countries()
    button = buttons[count]
    count += 1
    driver.execute_script("arguments[0].click();", button)

    try:#не у всех стран есть оено с городами
        open_window_all_cities()
    except Exception:
        pass
    
    time.sleep(1)

    #перезаход на сайт, чтобы взять новую страну
    driver.get("https://www.aviasales.by/")
    time.sleep(1.5)
    open_window_popular_countries()